# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS109A Introduction to Data Science 

## Lab 2 Scraping

**Harvard University**<br>
**Summer 2018**<br>
**Instructors:** Pavlos Protopapas and Kevin Rader <br>
**Lab Instructors:** Rahul Dave <br>
**Authors:** Rahul Dave, David Sondak, Will Claybaugh and Pavlos Protopaps

---

In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/cs109.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn.apionly as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

In [3]:
import time, requests

In this lab, we'll scrape Goodread's Best Books list:

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1 .

We'll walk through scraping the list pages for the book names/urls

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Exploring the Web pages and downloading them</li>
<li> Parse the page, extract book urls </li>
<li> Parse a book page, extract book properties </li>
<li> Set up a pipeline for fetching and parsing</li>
</ol>

## Learning Goals

Understand the structure of a web page. Use Beautiful soup to scrape content from these web pages.

*This lab corresponds to lectures 2, 3 and 4 and maps on to homework 1 and further.*

## 1. Exploring the web pages and downloading them

We're going to see the structure of Goodread's best books list. We'll use the Developer tools in chrome, safari and firefox have similar tools available

![](images/goodreads1.png)

To getch this page we use the `requests` module. But are we allowed to do this? Lets check:

https://www.goodreads.com/robots.txt

Yes we are.

In [4]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
url = URLSTART+BESTBOOKS+'1'
print(url)
page = requests.get(url)

https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1


We can see properties of the page. Most relevant are `status_code` and `text`. The former tells us  if the web-page was found, and if found , ok. (See lecture notes.)

In [5]:
page.status_code # 200 is good

200

In [6]:
page.text[:5000]

'<!DOCTYPE html>\n<html class="desktop\n">\n\n\n<head>\n  <title>\nBest Books Ever (54257 books)\n</title>\n\n\n    <script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();\n </script>\n  <script type="text/javascript">\n    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(m,l,h,j,i){var k={m:m,f:l,l:h,c:""+j,err:i,fromOnError:1,args:arguments};c.ueLogError(k);return false}b.skipTrace=1;e.onerror=b;function f(){c.uex("ld")}if(e.addEventListener){e.addEventListener("load",f,false)}else{if(e.attachEvent){e.attachEvent("onload",f)}}a.tag=d("tag");a.log=d("log");a.reset=d("rst");c.ue_csm=c;c.ue=a;c.ueLogError=d("err");c.ues=d("ues");c.uet=d("uet");c.uex=d("uex");c.uet("ue")})(window);(function(e,d){var a=e.ue||{};function c(g){if(!g){return}var f=d.head||d.getElementsByTagName("head")[0]||d.documentElem

Let us write a loop to fetch 2 pages of "best-books" from goodreads. Notice the use of a format string. This is an example of old-style python format strings

In [7]:
URLSTART="https://www.goodreads.com"
BESTBOOKS="/list/show/1.Best_Books_Ever?page="
for i in range(1,3):
    bookpage=str(i)
    stuff=requests.get(URLSTART+BESTBOOKS+bookpage)
    filetowrite="files/page"+ '%02d' % i + ".html"
    print("FTW", filetowrite)
    fd=open(filetowrite,"w")
    fd.write(stuff.text)
    fd.close()
    time.sleep(2)

FTW files/page01.html
FTW files/page02.html


## 2. Parse the page, extract book urls

Notice how we do file input-output, and use beautiful soup in the code below. The `with` construct ensures that the file being read is closed, something we do explicitly for the file being written. We look for the elements with class `bookTitle`, extract the urls, and write them into a file

In [8]:
from bs4 import BeautifulSoup

In [9]:
bookdict={}
for i in range(1,3):
    books=[]
    stri = '%02d' % i
    filetoread="files/page"+ stri + '.html'
    print("FTW", filetoread)
    with open(filetoread) as fdr:
        data = fdr.read()
    soup = BeautifulSoup(data, 'html.parser')
    for e in soup.select('.bookTitle'):
        books.append(e['href'])
    print(books[:10])
    bookdict[stri]=books
    fd=open("files/list"+stri+".txt","w")
    fd.write("\n".join(books))
    fd.close()

FTW files/page01.html
['/book/show/2767052-the-hunger-games', '/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix', '/book/show/2657.To_Kill_a_Mockingbird', '/book/show/1885.Pride_and_Prejudice', '/book/show/41865.Twilight', '/book/show/19063.The_Book_Thief', '/book/show/11127.The_Chronicles_of_Narnia', '/book/show/7613.Animal_Farm', '/book/show/18405.Gone_with_the_Wind', '/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set']
FTW files/page02.html
['/book/show/5470.1984', '/book/show/4989.The_Red_Tent', '/book/show/37435.The_Secret_Life_of_Bees', '/book/show/5.Harry_Potter_and_the_Prisoner_of_Azkaban', '/book/show/7171637-clockwork-angel', '/book/show/2187.Middlesex', '/book/show/2623.Great_Expectations', '/book/show/24583.The_Adventures_of_Tom_Sawyer', '/book/show/49552.The_Stranger', '/book/show/16299.And_Then_There_Were_None']


Here is George Orwell's 1984

In [10]:
bookdict['02'][0]

'/book/show/5470.1984'

 Lets go look at the first URLs on both pages

![](images/goodreads2.png)

## 3. Parse a book page, extract book properties

Ok so now lets dive in and get one of these these files and parse them.

In [11]:
furl=URLSTART+bookdict['02'][0]
furl

'https://www.goodreads.com/book/show/5470.1984'

![](images/goodreads3.png)

In [12]:
fstuff=requests.get(furl)
print(fstuff.status_code)

200


In [13]:
d=BeautifulSoup(fstuff.text, 'html.parser')

In [14]:
d.select("meta[property='og:title']")[0]['content']

'1984'

Lets get everything we want...

In [15]:
d=BeautifulSoup(fstuff.text, 'html.parser')
print(
"title", d.select_one("meta[property='og:title']")['content'],"\n",
"isbn", d.select("meta[property='books:isbn']")[0]['content'],"\n",
"type", d.select("meta[property='og:type']")[0]['content'],"\n",
"author", d.select("meta[property='books:author']")[0]['content'],"\n",
"average rating", d.select_one("span.average").text,"\n",
"ratingCount", d.select("meta[itemprop='ratingCount']")[0]["content"],"\n",
"reviewCount", d.select_one("span.count")["title"]
)

title 1984 
 isbn 9780451524935 
 type books.book 
 author https://www.goodreads.com/author/show/3706.George_Orwell 
 average rating 4.16 
 ratingCount 2401361 
 reviewCount 53311


Ok, now that we know what to do, lets wrap our fetching into a proper script. So that we dont overwhelm their servers, we will only fetch 5 from each page, but you get the idea...

We'll segue of a bit to explore new style format strings. See https://pyformat.info for more info.

In [16]:
"list{:0>2}.txt".format(3)

'list03.txt'

In [17]:
a = "4"
b = 4
class Four:
    def __str__(self):
        return "Fourteen"
c=Four()

In [18]:
"The hazy cat jumped over the {} and {} and {}".format(a, b, c)

'The hazy cat jumped over the 4 and 4 and Fourteen'

## 4. Set up a pipeline for fetching and parsing

Ok lets get back to the fetching...

In [20]:
fetched=[]
for i in range(1,3):
    with open("files/list{:0>2}.txt".format(i)) as fd:
        counter=0
        for bookurl_line in fd:
            if counter > 4:
                break
            bookurl=bookurl_line.strip()
            stuff=requests.get(URLSTART+bookurl)
            filetowrite=bookurl.split('/')[-1]
            filetowrite="files/"+str(i)+"_"+filetowrite+".html"
            print("FTW", filetowrite)
            fd=open(filetowrite,"w")
            fd.write(stuff.text)
            fd.close()
            fetched.append(filetowrite)
            time.sleep(2)
            counter=counter+1
            
print(fetched)

FTW files/1_2767052-the-hunger-games.html
FTW files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html
FTW files/1_2657.To_Kill_a_Mockingbird.html
FTW files/1_1885.Pride_and_Prejudice.html
FTW files/1_41865.Twilight.html
FTW files/2_5470.1984.html
FTW files/2_4989.The_Red_Tent.html
FTW files/2_37435.The_Secret_Life_of_Bees.html
FTW files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html
FTW files/2_7171637-clockwork-angel.html
['files/1_2767052-the-hunger-games.html', 'files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html', 'files/1_2657.To_Kill_a_Mockingbird.html', 'files/1_1885.Pride_and_Prejudice.html', 'files/1_41865.Twilight.html', 'files/2_5470.1984.html', 'files/2_4989.The_Red_Tent.html', 'files/2_37435.The_Secret_Life_of_Bees.html', 'files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html', 'files/2_7171637-clockwork-angel.html']


Ok we are off to parse each one of the html pages we fetched. We have provided the skeleton of the code and the code to parse the year, since it is a bit more complex...see the difference in the screenshots above. 

In [21]:
import re
yearre = r'\d{4}'
def get_year(d):
    if d.select_one("nobr.greyText"):
        return d.select_one("nobr.greyText").text.strip().split()[-1][:-1]
    else:
        thetext=d.select("div#details div.row")[1].text.strip()
        rowmatch=re.findall(yearre, thetext)
        if len(rowmatch) > 0:
            rowtext=rowmatch[0].strip()
        else:
            rowtext="NA"
        return rowtext

<div class="exercise"><b>Exercise</b></div>

Your job is to fill in the code to get the genres.

In [22]:
def get_genres(d):
    # your code here
    genres=d.select("div.elementList div.left a")
    glist=[]
    for g in genres:
        glist.append(g['href'])
    return glist

In [23]:

listofdicts=[]
for filetoread in fetched:
    print(filetoread)
    td={}
    with open(filetoread) as fd:
        datext = fd.read()
    d=BeautifulSoup(datext, 'html.parser')
    td['title']=d.select_one("meta[property='og:title']")['content']
    td['isbn']=d.select_one("meta[property='books:isbn']")['content']
    td['booktype']=d.select_one("meta[property='og:type']")['content']
    td['author']=d.select_one("meta[property='books:author']")['content']
    td['rating']=d.select_one("span.average").text
    td['ratingCount']=d.select_one("meta[itemprop='ratingCount']")["content"]
    td['reviewCount']=d.select_one("span.count")["title"]
    td['year'] = get_year(d)
    td['file']=filetoread
    glist = get_genres(d)
    td['genres']="|".join(glist)
    listofdicts.append(td)

files/1_2767052-the-hunger-games.html
files/1_2.Harry_Potter_and_the_Order_of_the_Phoenix.html
files/1_2657.To_Kill_a_Mockingbird.html
files/1_1885.Pride_and_Prejudice.html
files/1_41865.Twilight.html
files/2_5470.1984.html
files/2_4989.The_Red_Tent.html
files/2_37435.The_Secret_Life_of_Bees.html
files/2_5.Harry_Potter_and_the_Prisoner_of_Azkaban.html
files/2_7171637-clockwork-angel.html


In [24]:
listofdicts[0]

{'author': 'https://www.goodreads.com/author/show/153394.Suzanne_Collins',
 'booktype': 'books.book',
 'file': 'files/1_2767052-the-hunger-games.html',
 'genres': '/genres/young-adult|/genres/fiction|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action',
 'isbn': '9780439023481',
 'rating': '4.33',
 'ratingCount': '5489618',
 'reviewCount': '160353',
 'title': 'The Hunger Games (The Hunger Games, #1)',
 'year': '2008'}

Finally lets write all this stuff into a csv file which we will use to do analysis.

In [25]:
df = pd.DataFrame.from_records(listofdicts)
df.head()

author    booktype                                               file                                             genres           isbn rating ratingCount reviewCount                                              title  year
0  https://www.goodreads.com/author/show/153394.S...  books.book              files/1_2767052-the-hunger-games.html  /genres/young-adult|/genres/fiction|/genres/sc...  9780439023481   4.33     5489618      160353            The Hunger Games (The Hunger Games, #1)  2008
1  https://www.goodreads.com/author/show/1077326....  books.book  files/1_2.Harry_Potter_and_the_Order_of_the_Ph...  /genres/fantasy|/genres/young-adult|/genres/fi...  9780439358071   4.48     2029746       33025  Harry Potter and the Order of the Phoenix (Har...  2003
2  https://www.goodreads.com/author/show/1825.Har...  books.book            files/1_2657.To_Kill_a_Mockingbird.html  /genres/classics|/genres/fiction|/genres/histo...  9780061120084   4.27     3722273       79047  To Kill a Mockingbird (To Kill a Mockingbird, #1)  1960
3  https://www.goodreads.com/author/show/1265.Jan...  books.book              files/1_1885.Pride_and_Prejudice.html  /genres/classics|/genres/fiction|/genres/roman...  9780679783268   4.25     2437506       53999                                Pride and Prejudice  1813
4  https://www.goodreads.com/author/show/941441.S...  books.book                        files/1_41865.Twilight.html  /genres/young-adult|/genres/fantasy|/genres/ro...  9780316015844   3.58     4261792       97784                            Twilight (Twilight, #1)  2005

In [26]:
df.to_csv("files/meta.csv", index=False, header=True)